<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/%E6%96%B0%E5%9E%8B%E3%82%B3%E3%83%AD%E3%83%8A%E3%82%A6%E3%82%A4%E3%83%AB%E3%82%B9%E6%84%9F%E6%9F%93%E7%97%87%E3%81%AE%E5%A0%B1%E5%91%8A%E6%95%B0%E6%8E%A8%E7%A7%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新型コロナウイルス感染症の報告数推移

2020年に世界中でパンデミックを引き落とした新型コロナウイルス感染症は、2025年、人々の記憶から急速に忘却されつつあり、「ニューノーマル」という言葉すら無くなるほど日常に回帰した。
<br>
しかし新型株が登場するなど、その脅威が消え去ったわけではない。忘れないためにも、感染者数の推移を定点観測する場所があっても良いのではないかと思い、このノートブックを作成する。

In [1]:
!pip install camelot-py
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00


In [2]:
import requests
import io
import re

from bs4 import BeautifulSoup
import camelot
import pdfplumber

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# 厚労省のページから、5類移行後の感染者数推移を取得
# 詳細: https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000121431_00086.html
BASE_URLS = ["https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000121431_00438.html",  # 2023年
             "https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000121431_00461.html",  # 2024年
             "https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000121431_00474.html"  # 2025年
             ]

# ページから PDF リンクを取得
pdf_links = []
for BASE_URL in BASE_URLS:
    resp = requests.get(BASE_URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.endswith(".pdf") and "content" in href:
            if href.startswith("http"):
                pdf_links.append(href)
            else:
                pdf_links.append("https://www.mhlw.go.jp" + href)

print(f"Found {len(pdf_links)} PDF files")

Found 116 PDF files


In [4]:
def extract_start_date(period_str, year_hint):
    """
    '8月4日～8月10日' のような文字列から開始日を抽出し datetime に変換
    year_hint: '2025年第32週' のような列から取得した西暦を補完
    Args:
        period_str (str): 期間を示す文字列（例: '8月4日～8月10日', '12月26日～1月1日'）。
        year_hint (int or str): 週番号を含む年のヒント（例: 2024）。この週が属する西暦です。

    Returns:
        pd.Timestamp or None: 抽出された開始日のPandas Timestampオブジェクト。
                              形式が一致しない場合やpd.NAの場合はNone。
    """
    if pd.isna(period_str):
        return None

    # 日付を抽出
    m = re.match(
        r"([0-9０-９]+)月([0-9０-９]+)日[～~](\s*[0-9０-９]+)月([0-9０-９]+)日",
        str(period_str)
    )
    if not m:
        return None

    start_month = int(m.group(1))
    start_day = int(m.group(2))
    end_month = int(m.group(3))

    year = int(year_hint)

    # 期間が年度をまたぐかどうかを判定し、必要に応じて年を調整
    if start_month == 12 and end_month == 1:
        year -= 1

    return pd.Timestamp(year=int(year), month=start_month, day=start_day)

In [ ]:
# 各PDFから都道府県別データを抽出
all_dfs = []
for link in pdf_links:
    print("Processing:", link)
    try:
        # PDFをダウンロード
        pdf_resp = requests.get(link)
        pdf_resp.raise_for_status()

        # 一時ファイル扱いにして読み込む
        with open("temp.pdf", "wb") as f:
            f.write(pdf_resp.content)

        # 2ページ目を抽出
        tables = camelot.read_pdf("temp.pdf", pages="2")
        if len(tables) == 0:
            print("No tables found")
            continue
        df = tables[0].df

        # 日付情報を取得
        with open("temp.pdf", "rb") as f:
            with pdfplumber.open(f) as pdf:
                page2_text = pdf.pages[1].extract_text()

        week_info_match = re.search(r"(\d{4})年第\s*(\d+)\s*週\(?([0-9月日〜\s\-～]+)\)?", page2_text)
        if week_info_match:
            year = int(week_info_match.group(1))
            week_no = int(week_info_match.group(2))
            period = week_info_match.group(3).strip() if week_info_match.group(3) else None
            week_label = f"{year}年第{week_no}週"
        else:
            print("No week info found")
            week_label, period = None, None

        # DataFrame調整
        df.columns = ['都道府県', '報告数', '定点あたり']

        if  df.shape[0] < 51:  # 「昨年同月比」レコードが追加されているか否か
            df = df.drop([0, 1, 49]).reset_index(drop=True)
        else:
            df = df.drop([0, 1, 49, 50]).reset_index(drop=True)

        df = df.replace("\n", "", regex=True)
        df["報告数"] = df["報告数"].str.replace(",", "").astype(int)
        df["定点あたり"] = df["定点あたり"].astype(float)
        df["週"] = week_label
        df["期間"] = period

        all_dfs.append(df)

    except Exception as e:
        print("Failed to process", link, e)

# 結合
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df["year"] = final_df["週"].str.extract(r"(\d{4})").astype(int)
    final_df["開始日"] = final_df.apply(lambda x: extract_start_date(x["期間"], x["year"]), axis=1)
    print(final_df.head())
else:
    final_df = pd.DataFrame()
    print("No tables extracted")

# 保存
final_df.to_csv("covid_prefecture_timeseries.csv", index=False, encoding="utf-8-sig")

In [ ]:
# 都道府県別に複数レコードがあるため、ここでは全国合計を例示
df_total = final_df.groupby("開始日", as_index=False)["報告数"].sum().sort_values("開始日")

# Plotlyで折れ線グラフ
fig = px.line(df_total, x="開始日", y="報告数", title="新型コロナ報告数（週次・全国合計）")
fig.show()